In [ ]:
from os import listdir
from pprint import pprint

import pdfplumber
import pandas as pd

from model.columns_pdf import ColumnsPdf
from model.pdf import Pdf

In [ ]:
def read_pdf(path: str):
    with pdfplumber.open(path) as pdf:
        texts = []
        table = []
        for page in pdf.pages:
            extracted = page.extract_text()
            if extracted is None:
                continue
            texts += filter(lambda i: i, extracted.split("\n"))
            table += page.extract_table(table_settings={
                "vertical_strategy": "lines",
                "horizontal_strategy": "text",
                "snap_tolerance": 2,
            }) or []
    return texts, table

# DEBUG
texts, table = read_pdf("CDMATR~1.PDF")
pd.DataFrame(
    data=table
)

In [ ]:
def convert_table(texts: list, table: list):
    list_lines = []
    list_exists = []
    for line in table:
        for j, text in enumerate(texts):
            if line[0] in text:
                if j in list_exists:
                    continue
                else:
                    list_exists.append(j)
                
                columns_pdf = ColumnsPdf()
                columns_pdf.cci = text.split()[0] if text.split()[0].isdigit() else ""
                columns_pdf.descricao = line[0]
                columns_pdf.leitura_alterior = line[1]
                columns_pdf.leitura_atual = line[2]
                columns_pdf.registrado = line[3]
                columns_pdf.faturado = line[4]
                columns_pdf.tarifa_com_icms = line[5]
                columns_pdf.base_icms = line[6]
                columns_pdf.aliq_icms = line[7]
                columns_pdf.icms = line[8]
                columns_pdf.valor = line[9]
                columns_pdf.tarifa_sem_impostos = text.split()[-1] if line[-1] and text.split()[-1] != line[-1] else ""
                list_lines.append(columns_pdf)
                break
    return list_lines

# DEBUG
# TODO

In [ ]:
def convert(texts: list, table: list):
    pdf = Pdf()

    numbers = texts[2].split(' ')
    pdf.n_instalacao, pdf.n_cliente, pdf.n_medidor = numbers[0], numbers[1], numbers[2]
    pdf.cnpj = texts[3].split('CPF/CNPJ:')[1].split(' ')[1]
    pdf.endereco = texts[6]
    for text in texts:
        if 'CEP' in text:
            pdf.cep = text.split(' ')[1]
            break

    pdf.tabela = convert_table(texts, table)
    return pdf

# DEBUG
# TODO
# table.insert(33, ['PIS/PASEP (0,59%)', '', '', '', '', '', '0,00', '0%', '0,00', '77,21-']) # teste
# texts[0].insert(54, '0699 PIS/PASEP (0,59%) 0,00 0% 0,00 77,27-') # teste

In [ ]:
def full(path: str):
    texts, table = read_pdf(path)
    pdf = convert(texts, table)
    return pdf

# DEBUG
example = full("CDMATR~1.PDF")
# example.to_dataframe()
example.tabela_to_dataframe()
pd.DataFrame(
    [
        ["Pedro", 2.3],
        ["Cuca", 0.2]
    ], columns=["nome", "kd"]
)

# TODO:
- Adicionar `Data Emissão`;
- Adicionar `Conta Referente a`;
- Adicionar `Vencimento`;